In [1]:
import os
import nltk
import numpy as np
import copy

In [2]:
nltk.download('punkt')

documents = {}
labels = {}

# open main data
for file in os.listdir("/Users/dunbanghe/Desktop/Hulth2003/Training"):
    if file.endswith(".abstr"):
        content = open(("%s/%s" % ('/Users/dunbanghe/Desktop/Hulth2003/Training', file)), "r").read()
        documents[file.split('.')[0]] = content.split('. ')
        
# open labels        
for file in os.listdir("/Users/dunbanghe/Desktop/Hulth2003/Training"):
    if file.endswith(".uncontr"):
        content = open(("%s/%s" % ('/Users/dunbanghe/Desktop/Hulth2003/Training', file)), "r").read()
        labels[file.split('.')[0]] = content.split("; ")       

[nltk_data] Downloading package punkt to /Users/dunbanghe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#labels

In [4]:
# tokenize document
tokenized_documents = {}
for num, ctt in documents.items():
    tokenized_documents[num] = []
    for sentence in ctt:
        words = nltk.word_tokenize(sentence.lower())
        #words=[word.lower() for word in words if word.isalpha()]
        tokenized_documents[num].append(words)

# tokenize labels
tokenized_labels = {}
for num, ctt in labels.items():
    tokenized_labels[num] = []
    for sentence in ctt:
        words = nltk.word_tokenize(sentence.lower())
        #words=[word.lower() for word in words if word.isalpha()]
        tokenized_labels[num].append(words)        

In [5]:
#PROBLEM 
# For document 1346 keyphrase should be "growing time adaptive self-organizing map", but word self-organized is lost
# Fixed

In [6]:
tokenized_labels['1346']

[['automatic', 'multilevel', 'thresholding'],
 ['image', 'segmentation'],
 ['growing', 'time', 'adaptive', 'self-organizing', 'map'],
 ['growing', 'tasom'],
 ['gtasom'],
 ['peak', 'finding', 'process']]

In [7]:
# function find index of element in list of lists
def index(lab, target):
    for i,phr in enumerate(lab):
        for j, w in enumerate(phr):
            if w == target:
                return (j)
    return (None, None)

# create dictionary of labels associated to words
class_labels = copy.deepcopy(tokenized_documents)
for document in tokenized_documents:
    text = tokenized_documents[document]
    lab = tokenized_labels[document]
    lab_flattened = [val for sublist in lab for val in sublist]
    for i, sentence in enumerate(text): 
        for j, word in enumerate(sentence): 
            is_keyphrase = word in lab_flattened
            if is_keyphrase:
                if index(tokenized_labels[document], word) == 0:
                    class_labels[document][i][j] = 1
                else:
                    class_labels[document][i][j] = 2
            else:
                class_labels[document][i][j] = 0

# GLOVE embeddings

In [8]:
import numpy as np
embeddings_index_100 = dict()
f = open('glove.6B/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index_100[word] = coefs
f.close()

FileNotFoundError: [Errno 2] No such file or directory: 'glove.6B/glove.6B.100d.txt'

In [ ]:
embeddings_index_100

In [ ]:
tokenized_documents['1346']